# Creating DB Model in pgAdmin 4

<p>First, it is usefull to look into the data file. Typically it means examining the structure, identifying possible problems such as missing data or different data types for particuluar columns. In this case, it was obvious that data contains nested objects. Therefore, I expected to execute preprocessing and reformating data.

## Table structure

<p>After initial examination of data, I create simple data model. I prefer to use dbdiagram for creating the model</p>

<b>Visually it looks like this:

![Example Image](data_model.png)

<p>Now, when my model is ready, I can start to create tables in my db. I log to pgAdmin4 and create following table structure</p>

<b> As you can see, I transformed my model a little bit. As I'm not using quantity as I assumed previously I altered the table to not have pk.

## Inserting data to tables

<p>Now my model is ready. I need to create python script to insert data from json file into my tables My json file contains nested structures, thus I must extract data from nested objects and insert it into correct tables while removing duplicates.  </p>
<I>Note: For now I'm keeping the column created intact. I'm adding column quantity in case one product is present in customer order multiple times (standard situation)</I>


<b>Python script for entering the data:

In [ ]:
import json
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(
    host='localhost',
    database='postgres',
    user='postgres',
    password='*******',
    port="5432"
)

cur = conn.cursor()

try:
    # Read the ndjson file and insert data into the database
    with open('dataset/data.ndjson', 'r') as file:
        for line in file:
            data = json.loads(line)

            # Extract data from the JSON structure
            order_id = data['id']
            created = data['created']
            user_id = data['user']['id']

            # Insert data into Customers table
            cur.execute("""
                INSERT INTO Customers (user_id, user_name, user_city)
                VALUES (%s, %s, %s)
                ON CONFLICT (user_id) DO NOTHING;
            """, (user_id, data['user']['name'], data['user']['city']))

            # Insert data into Products table and create a separate row for each product
            for i, product in enumerate(data['products']):
                product_id = product['id']
                cur.execute("""
                    INSERT INTO Products (product_id, product_name, product_price)
                    VALUES (%s, %s, %s)
                    ON CONFLICT (product_id) DO NOTHING;
                """, (product_id, product['name'], product['price']))

                # Insert data into Orders table for each product
                cur.execute("""
                    INSERT INTO Orders (order_id, created, customer_id, product_id)
                    VALUES (%s, %s, %s, %s);
                """, (order_id, created, user_id, product_id))

    conn.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
    cur.close()
    conn.close()


## Adjust data in tables

<p>Now, I have data ready in my tables. After quick check, I can see that my data are located it correct tables and don't contain missing or duplicate data. Now, I updated column created and change datatype to timestamp. After this, I'am able to create new sql query for returning only fridays.</p>

<b>Change column created to TIMESTAMP:

<b>Filter for Fridays

## Answering the Questions

### Question 1: Which user spent the most money on products on all Fridays?

<p>In this case, I must create a query that will return only orders executed on Fridays. I need to know which user executed this order. I also need to calculate cumulative amount of money spent on one order and then sum up all orders executed on fridays by this specific user.</p><b> Thus my solutions looks like this:</b>

In [9]:
WITH FridayOrders AS (
    SELECT
        orders.customer_id,
        products.product_price  AS spending
    FROM
        Orders
        JOIN products ON orders.product_id = products.product_id
    WHERE
        EXTRACT(DOW FROM orders.created) = 5 -- Filter for Fridays
)

SELECT
    customers.user_id,
    customers.user_name,
    SUM(FridayOrders.spending) AS total_spending
FROM
    FridayOrders
    JOIN Customers ON FridayOrders.customer_id = customers.user_id
GROUP BY
    customers.user_id, customers.user_name
ORDER BY
    total_spending DESC  total_spending DESC

SyntaxError: invalid syntax (3948633944.py, line 1)

<b>The following result is presented:

In [15]:
import pandas as pd
df = pd.read_csv('results/question1.csv')
df

,user_id,user_name,total_spending
0,4,User E,869810
1,1,User B,862980
2,5,User F,855470
3,2,User C,835880
4,3,User D,831690
5,7,User H,825720
6,6,User G,825350
7,0,User A,814700


### Answer question 1:
<b>The answer is: USER E with user id: 4 and total spending of 869810 spend the most money on products on all Fridays

### Question 2: What are the best 3 products in each location of a user based on quantity?

<p>In this case, I must create a query that will return 3 products for each location users executed the purchase.The main criterium is quantity. I need to calculate the total quantity of each product for each user in each city, and assign a rank to each product based on quantity within the user and city.</p><b> Thus my solutions looks like this:</b>

WITH UserProductRank AS (
    SELECT
        customers.user_id,
        customers.user_city,
        products.product_name,
        SUM(orders.quantity) AS total_quantity,
        RANK() OVER (PARTITION BY customers.user_id, customers.user_city ORDER BY SUM(orders.quantity) DESC) AS ranking
    FROM
        customers
        JOIN orders ON customers.user_id = orders.customer_id
        JOIN products ON orders.product_id = products.product_id
    GROUP BY
        customers.user_id, customers.user_city, products.product_name
)

SELECT
    user_id,
    user_city,
    product_name,
    total_quantity
FROM
    UserProductRank
WHERE
    ranking <= 3;

<b>The following result is presented:

In [16]:
import pandas as pd
df = pd.read_csv('results/question2.csv')
df

,user_id,user_city,product_name,total_quantity
0,0,Prague,Product C,4400
1,0,Prague,Product L,4388
2,0,Prague,Product G,4362
3,1,Brno,Product L,4438
4,1,Brno,Product O,4413
5,1,Brno,Product N,4386
6,2,Singapore,Product P,4479
7,2,Singapore,Product N,4396
8,2,Singapore,Product C,4381
9,3,Sydney,Product M,4335


### Answer question 2:
<b>Based on the results shown above it is clear which products are the best products for a particular location. For example: for location Prague it is Product C with the quantity of 4400, followed by Product L and Product G, while for location Brno it is Product L with the quantity of 4438, followed by Product O and Product N and etc.